In [1]:
!cat .env

STAGE=PRODUCTION


In [2]:
from copy import deepcopy
from tqdm.notebook import tqdm
from django.contrib.admin.utils import flatten
from helpers.strings import keys_to_snake_case, keys_to_camel_case
from rest_framework_json_api.utils import format_field_names

In [3]:
def format_keys(obj, format_type="camelize"):
    if isinstance(obj, dict):
        for key, value in obj.items():
            if isinstance(value, dict):
                obj[key] = format_field_names(
                            format_keys(value), format_type
                            )
            elif isinstance(value, list):
                list_temp = value.copy()
                for i, item in enumerate(value):
                    if isinstance(item, dict):
                        list_temp[i] = format_field_names(
                        format_keys(item), format_type
                        )
                    else:
                        list_temp[i] = item
                obj[key] = list_temp
    return obj

In [5]:
company = Company.objects.get(name="Arteris Via Paulista")
company

<Company: 433c0cd2-80e2-46c9-b8c5-030fc61070e5: Arteris Via Paulista>

In [12]:
occurrence = OccurrenceType.objects.filter(company=company).order_by('name')
occurrence.count()

268

In [7]:
for match in tqdm(occurrence):
    for index, field in enumerate(match.form_fields['fields']):
        match.form_fields['fields'][index]=format_keys(keys_to_camel_case(field))
    for indexg, field_group in enumerate(match.form_fields['groups']):
        match.form_fields['groups'][indexg]=format_keys(keys_to_camel_case(field_group))
    match.save()

  0%|          | 0/268 [00:00<?, ?it/s]

In [11]:
for ac in tqdm(occurrence):
    size = len(ac.form_fields['fields'])
    for arr in ac.form_fields['fields']:
        if 'demandOrigin' == arr.get('apiName'):
            try:
                dop = next(a['value'] for a in arr['selectOptions']['options'] if a['name']=='ARTESP DOP')
                kcor = next(a['value'] for a in arr['selectOptions']['options'] if a['name']=='KCOR')
                print(dop,kcor,len(arr['selectOptions']['options']),ac.name,ac.occurrence_kind)
            except Exception as e:
                print(e)

  0%|          | 0/268 [00:00<?, ?it/s]

4 2 7 Aceiros 701
4 2 7 Animais mortos 702
4 2 7 Atividade – Reposição (baixa refletância/furto/avariada)(solo/aérea) 2002
4 2 7 ATR 131
4 2 7 Ausência - falta; inexistência 12
4 2 7 Avariada - danificada - deteriorada; abalroada; avariada; vandalizada; amassada 12
4 2 7 Baixa visibiliade noturna ou película inadequada ou retro inadequada 12
4 2 7 Bordos e lajes quebrados 6
4 2 7 Call box - base; acesso; proteção; local inadequado 12
4 2 7 Capina 701
4 2 8 Carreamento 15
4 2 7 Ciclovia - irregular; desgastada; mal sinalizada 12




4 2 7 Conflitante - apagada com tinta preta; sobreposta 12
4 2 7 Conformação lateral de drenagem 801
4 2 8 Contaminação de solo 15
4 2 7 Corte e poda de árvores 701
4 2 7 Degrau junto ao pavimento 706
4 2 7 Degrau no Acostamento 13

4 2 7 Depressões 13
4 2 7 Despraguejamento 701
4 2 7 Encoberta - pela vegetação; encoberta por outra placa (no caso de vertical) 12
4 2 7 Ensaio pavimento 8

4 2 7 Erosões de aterro 703
4 2 7 Erosões de corte 703
4 2 8 Escorregam

In [13]:
for ac in tqdm(occurrence):
#     size = len(ac.form_fields['fields'])
    for arr in ac.form_fields['fields']:
        if arr.get('apiName') == 'internCode':
            arr['displayIf']['or'].append({
            "in": [
              "4",
              {
                "var": "data.formData.demandOrigin"
              }
            ]
          })
            arr['displayIf']['or'].append({
            "in": [
              "2",
              {
                "var": "data.formData.demandOrigin"
              }
            ]
          })
    ac.save()

  0%|          | 0/268 [00:00<?, ?it/s]

# Verificar se o campo existe no groups para inserção no inicio da lista e reordenação dos orders seguintes

In [51]:
tem_id=[]
for ac in tqdm(occurrence):
    if any(d.get('displayName') == 'Identificação' for d in ac.form_fields['groups']):
        tem_id.append(ac)
len(tem_id)

  0%|          | 0/149 [00:00<?, ?it/s]

102

In [52]:
occurrence.exclude(name__in=[a.name for a in tem_id]).count()+len(tem_id)

149

# Voltar antes da alteração feita

In [8]:
for ac in tqdm(occurrence):
    hist = ac.history.all().order_by('-history_date')[1]
    hist.instance.save()

  0%|          | 0/230 [00:00<?, ?it/s]

In [9]:
occurrence[0],occurrence[0].occurrence_kind

(<OccurrenceType: Aceiro - ['Arteris Planalto Sul - Conserva']>, '81')

# logica de municipios arteris

In [33]:
for ac in tqdm(occurrence):
    size = len(ac.form_fields['fields'])
    teste=True
    for arr in ac.form_fields['fields']:
        if 'city' == arr.get('apiName'):
            teste = False
            arr['autofill'] = {"if": [{"and": [{"==": [{"var": "roadName"}, "SP-249"]}, {">=": [{"var": "km"}, 144.115]}, {"<=": [{"var": "km"}, 149.0]}]}, "Coronel Macedo", {"and": [{"==": [{"var": "roadName"}, "SP-249"]}, {">=": [{"var": "km"}, 149.0]}, {"<=": [{"var": "km"}, 159.0]}]}, "Taquarituba", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 83.2]}, {"<=": [{"var": "km"}, 95.16]}]}, "Araraquara", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 95.16]}, {"<=": [{"var": "km"}, 122.22]}]}, "Boa Esperan\u00e7a do Sul", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 122.22]}, {"<=": [{"var": "km"}, 124.02]}]}, "Dourado", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 124.02]}, {"<=": [{"var": "km"}, 138.61]}]}, "Bocaina", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 138.61]}, {"<=": [{"var": "km"}, 147.3]}]}, "Ja\u00fa", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 155.77]}, {"<=": [{"var": "km"}, 167.02]}]}, "Ja\u00fa", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 167.02]}, {"<=": [{"var": "km"}, 177.8]}]}, "Barra Bonita", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 177.8]}, {"<=": [{"var": "km"}, 186.7]}]}, "Igara\u00e7u do Tiet\u00ea", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 186.7]}, {"<=": [{"var": "km"}, 212.24]}]}, "S\u00e3o Manuel", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 212.24]}, {"<=": [{"var": "km"}, 225.83]}]}, "Prat\u00e2nia", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 225.83]}, {"<=": [{"var": "km"}, 237.06]}]}, "Botucatu", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 237.06]}, {"<=": [{"var": "km"}, 237.77]}]}, "Avar\u00e9", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 288.19]}, {"<=": [{"var": "km"}, 309.04]}]}, "Ita\u00ed", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 309.04]}, {"<=": [{"var": "km"}, 320.0]}]}, "Taquarituba", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 334.25]}, {"<=": [{"var": "km"}, 345.46]}]}, "Coronel Macedo", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 345.46]}, {"<=": [{"var": "km"}, 357.5]}]}, "Itaporanga", {"and": [{"==": [{"var": "roadName"}, "SP-281"]}, {">=": [{"var": "km"}, 43.36]}, {"<=": [{"var": "km"}, 49.64]}]}, "Riversul", {"and": [{"==": [{"var": "roadName"}, "SP-281"]}, {">=": [{"var": "km"}, 49.64]}, {"<=": [{"var": "km"}, 67.72]}]}, "Itaporanga", {"and": [{"==": [{"var": "roadName"}, "SP-281"]}, {">=": [{"var": "km"}, 67.72]}, {"<=": [{"var": "km"}, 70.85]}]}, "Bar\u00e3o de Antonina", {"and": [{"==": [{"var": "roadName"}, "SP-304"]}, {">=": [{"var": "km"}, 285.0]}, {"<=": [{"var": "km"}, 297.0]}]}, "Ja\u00fa", {"and": [{"==": [{"var": "roadName"}, "SPA-043/281"]}, {"<=": [{"var": "km"}, 3.0]}]}, "Riversul", {"and": [{"==": [{"var": "roadName"}, "SPA-085/255"]}, {"<=": [{"var": "km"}, 1.115]}]}, "Araraquara", {"and": [{"==": [{"var": "roadName"}, "SPA-106/255"]}, {"<=": [{"var": "km"}, 7.25]}]}, "Boa Esperan\u00e7a do Sul", {"and": [{"==": [{"var": "roadName"}, "SPA-112/255"]}, {"<=": [{"var": "km"}, 1.1]}]}, "Boa Esperan\u00e7a do Sul", {"and": [{"==": [{"var": "roadName"}, "SPA-114/255"]}, {"<=": [{"var": "km"}, 1.0]}]}, "Boa Esperan\u00e7a do Sul", {"and": [{"==": [{"var": "roadName"}, "SPA-115/255"]}, {"<=": [{"var": "km"}, 2.1]}]}, "Boa Esperan\u00e7a do Sul", {"and": [{"==": [{"var": "roadName"}, "SPA-115/255"]}, {">=": [{"var": "km"}, 2.1]}, {"<=": [{"var": "km"}, 7.2]}]}, "Trabiju", {"and": [{"==": [{"var": "roadName"}, "SPA-133/255"]}, {"<=": [{"var": "km"}, 5.7]}]}, "Bocaina", {"and": [{"==": [{"var": "roadName"}, "SPA-138/255"]}, {"<=": [{"var": "km"}, 4.8]}]}, "Bocaina", {"and": [{"==": [{"var": "roadName"}, "SPA-179/255"]}, {"<=": [{"var": "km"}, 0.4]}]}, "Igara\u00e7u do Tiet\u00ea", {"and": [{"==": [{"var": "roadName"}, "SPA-196/255"]}, {"<=": [{"var": "km"}, 2.5]}]}, "S\u00e3o Manuel", {"and": [{"==": [{"var": "roadName"}, "SPA-198/255"]}, {"<=": [{"var": "km"}, 5.7]}]}, "S\u00e3o Manuel", {"and": [{"==": [{"var": "roadName"}, "SPA-214/255"]}, {"<=": [{"var": "km"}, 0.7]}]}, "Prat\u00e2nia", {"and": [{"==": [{"var": "roadName"}, "SPA-338/255"]}, {"<=": [{"var": "km"}, 0.93]}]}, "Coronel Macedo", {"and": [{"==": [{"var": "roadName"}, "SPI-274/310"]}, {"<=": [{"var": "km"}, 2.1]}]}, "Araraquara", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 2.8]}, {"<=": [{"var": "km"}, 19.0]}]}, "Ribeir\u00e3o Preto", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 19.0]}, {"<=": [{"var": "km"}, 24.25]}]}, "Cravinhos", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 24.25]}, {"<=": [{"var": "km"}, 32.87]}]}, "Lu\u00eds Ant\u00f4nio", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 32.87]}, {"<=": [{"var": "km"}, 47.6]}]}, "Guatapar\u00e1", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 47.6]}, {"<=": [{"var": "km"}, 53.5]}]}, "Rinc\u00e3o", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 53.5]}, {"<=": [{"var": "km"}, 61.1]}]}, "Santa L\u00facia", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 61.1]}, {"<=": [{"var": "km"}, 71.0]}]}, "Am\u00e9rico Brasiliense", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 71.0]}, {"<=": [{"var": "km"}, 83.2]}]}, "Araraquara", {"and": [{"==": [{"var": "roadName"}, "SP-257"]}, {"<=": [{"var": "km"}, 6.92]}]}, "Am\u00e9rico Brasiliense", {"and": [{"==": [{"var": "roadName"}, "SP-257"]}, {">=": [{"var": "km"}, 6.92]}, {"<=": [{"var": "km"}, 16.1]}]}, "Santa L\u00facia", {"and": [{"==": [{"var": "roadName"}, "SP-257"]}, {">=": [{"var": "km"}, 16.1]}, {"<=": [{"var": "km"}, 19.8]}]}, "Rinc\u00e3o", {"and": [{"==": [{"var": "roadName"}, "SP-318"]}, {">=": [{"var": "km"}, 235.4]}, {"<=": [{"var": "km"}, 267.4]}]}, "S\u00e3o Carlos", {"and": [{"==": [{"var": "roadName"}, "SP-318"]}, {">=": [{"var": "km"}, 267.4]}, {"<=": [{"var": "km"}, 270.5]}]}, "Am\u00e9rico Brasiliense", {"and": [{"==": [{"var": "roadName"}, "SP-318"]}, {">=": [{"var": "km"}, 270.5]}, {"<=": [{"var": "km"}, 271.6]}]}, "Santa L\u00facia", {"and": [{"==": [{"var": "roadName"}, "SP-318"]}, {">=": [{"var": "km"}, 271.6]}, {"<=": [{"var": "km"}, 280.0]}]}, "Rinc\u00e3o", {"and": [{"==": [{"var": "roadName"}, "SP-328"]}, {">=": [{"var": "km"}, 289.83]}, {"<=": [{"var": "km"}, 292.76]}]}, "S\u00e3o Sim\u00e3o", {"and": [{"==": [{"var": "roadName"}, "SP-328"]}, {">=": [{"var": "km"}, 292.76]}, {"<=": [{"var": "km"}, 300.28]}]}, "Cravinhos", {"and": [{"==": [{"var": "roadName"}, "SP-328"]}, {">=": [{"var": "km"}, 300.28]}, {"<=": [{"var": "km"}, 300.964]}]}, "Ribeir\u00e3o Preto", {"and": [{"==": [{"var": "roadName"}, "SP-328"]}, {">=": [{"var": "km"}, 300.964]}, {"<=": [{"var": "km"}, 301.518]}]}, "Cravinhos", {"and": [{"==": [{"var": "roadName"}, "SP-328"]}, {">=": [{"var": "km"}, 301.518]}, {"<=": [{"var": "km"}, 302.06]}]}, "Ribeir\u00e3o Preto", {"and": [{"==": [{"var": "roadName"}, "SP-328"]}, {">=": [{"var": "km"}, 304.31]}, {"<=": [{"var": "km"}, 306.82]}]}, "Ribeir\u00e3o Preto", {"and": [{"==": [{"var": "roadName"}, "SP-328"]}, {">=": [{"var": "km"}, 307.6]}, {"<=": [{"var": "km"}, 311.93]}]}, "Ribeir\u00e3o Preto", {"and": [{"==": [{"var": "roadName"}, "SP-330"]}, {">=": [{"var": "km"}, 240.0]}, {"<=": [{"var": "km"}, 261.0]}]}, "Santa Rita do Passa Quatro", {"and": [{"==": [{"var": "roadName"}, "SP-330"]}, {">=": [{"var": "km"}, 261.0]}, {"<=": [{"var": "km"}, 284.81]}]}, "S\u00e3o Sim\u00e3o", {"and": [{"==": [{"var": "roadName"}, "SP-330"]}, {">=": [{"var": "km"}, 284.81]}, {"<=": [{"var": "km"}, 289.65]}]}, "Cravinhos", {"and": [{"==": [{"var": "roadName"}, "SP-330"]}, {">=": [{"var": "km"}, 289.65]}, {"<=": [{"var": "km"}, 300.34]}]}, "Cravinhos", {"and": [{"==": [{"var": "roadName"}, "SP-330"]}, {">=": [{"var": "km"}, 300.34]}, {"<=": [{"var": "km"}, 319.0]}]}, "Ribeir\u00e3o Preto", {"and": [{"==": [{"var": "roadName"}, "SP-334"]}, {">=": [{"var": "km"}, 318.0]}, {"<=": [{"var": "km"}, 321.0]}]}, "Ribeir\u00e3o Preto", {"and": [{"==": [{"var": "roadName"}, "SP-334"]}, {">=": [{"var": "km"}, 321.0]}, {"<=": [{"var": "km"}, 330.96]}]}, "Jardin\u00f3polis", {"and": [{"==": [{"var": "roadName"}, "SP-334"]}, {">=": [{"var": "km"}, 330.96]}, {"<=": [{"var": "km"}, 342.05]}]}, "Brod\u00f3wski", {"and": [{"==": [{"var": "roadName"}, "SP-334"]}, {">=": [{"var": "km"}, 342.05]}, {"<=": [{"var": "km"}, 372.0]}]}, "Batatais", {"and": [{"==": [{"var": "roadName"}, "SP-334"]}, {">=": [{"var": "km"}, 372.0]}, {"<=": [{"var": "km"}, 388.65]}]}, "Restinga", {"and": [{"==": [{"var": "roadName"}, "SP-334"]}, {">=": [{"var": "km"}, 388.65]}, {"<=": [{"var": "km"}, 409.0]}]}, "Franca", {"and": [{"==": [{"var": "roadName"}, "SP-345"]}, {">=": [{"var": "km"}, 10.5]}, {"<=": [{"var": "km"}, 12.2]}]}, "Itirapu\u00e3", {"and": [{"==": [{"var": "roadName"}, "SP-345"]}, {">=": [{"var": "km"}, 10.0]}, {"<=": [{"var": "km"}, 25.5]}]}, "Patroc\u00ednio Paulista", {"and": [{"==": [{"var": "roadName"}, "SP-345"]}, {">=": [{"var": "km"}, 25.5]}, {"<=": [{"var": "km"}, 40.0]}]}, "Franca", {"and": [{"==": [{"var": "roadName"}, "SPA-004/257"]}, {"<=": [{"var": "km"}, 4.6]}]}, "Am\u00e9rico Brasiliense", {"and": [{"==": [{"var": "roadName"}, "SPA-011/345"]}, {"<=": [{"var": "km"}, 1.8]}]}, "Itirapu\u00e3", {"and": [{"==": [{"var": "roadName"}, "SPA-020/345"]}, {"<=": [{"var": "km"}, 2.5]}]}, "Patroc\u00ednio Paulista", {"and": [{"==": [{"var": "roadName"}, "SPA-051/255"]}, {"<=": [{"var": "km"}, 12.8]}]}, "Rinc\u00e3o", {"and": [{"==": [{"var": "roadName"}, "SPA-074/255"]}, {"<=": [{"var": "km"}, 5.0]}]}, "Araraquara", {"and": [{"==": [{"var": "roadName"}, "SPA-077/255"]}, {"<=": [{"var": "km"}, 3.0]}]}, "Araraquara", {"and": [{"==": [{"var": "roadName"}, "SPA-241/330"]}, {"<=": [{"var": "km"}, 11.95]}]}, "Santa Rita do Passa Quatro", {"and": [{"==": [{"var": "roadName"}, "SPA-268/310"]}, {"<=": [{"var": "km"}, 2.9]}]}, "Araraquara", {"and": [{"==": [{"var": "roadName"}, "SPA-271/310"]}, {"<=": [{"var": "km"}, 0.58]}]}, "Araraquara", {"and": [{"==": [{"var": "roadName"}, "SPA-307/330"]}, {"<=": [{"var": "km"}, 3.6]}]}, "Ribeir\u00e3o Preto", {"and": [{"==": [{"var": "roadName"}, "SPA-318/330"]}, {"<=": [{"var": "km"}, 1.65]}]}, "Ribeir\u00e3o Preto", {"and": [{"==": [{"var": "roadName"}, "SPA-321/334"]}, {"<=": [{"var": "km"}, 4.3]}]}, "Jardin\u00f3polis", {"and": [{"==": [{"var": "roadName"}, "SPA-334/334"]}, {"<=": [{"var": "km"}, 1.469]}]}, "Brod\u00f3wski", {"and": [{"==": [{"var": "roadName"}, "SPA-334/334"]}, {">=": [{"var": "km"}, 1.469]}, {"<=": [{"var": "km"}, 9.7]}]}, "Jardin\u00f3polis", {"and": [{"==": [{"var": "roadName"}, "SPA-348/334"]}, {"<=": [{"var": "km"}, 3.5]}]}, "Batatais", {"and": [{"==": [{"var": "roadName"}, "SPA-352/334"]}, {"<=": [{"var": "km"}, 2.9]}]}, "Batatais", {"and": [{"==": [{"var": "roadName"}, "SPA-384/334"]}, {"<=": [{"var": "km"}, 4.2]}]}, "Restinga", {"and": [{"==": [{"var": "roadName"}, "SPA-397/334"]}, {">=": [{"var": "km"}, 2.44]}, {"<=": [{"var": "km"}, 4.47]}]}, "Franca", {"and": [{"==": [{"var": "roadName"}, "SPA-397/334"]}, {">=": [{"var": "km"}, 4.47]}, {"<=": [{"var": "km"}, 9.95]}]}, "Restinga", "NA"]}
    if teste:
        ac.form_fields['fields'].append({"id": size+1,
                                        "apiName": "city",
                                        "dataType": "string",
                                        "autofill":{"if": [{"and": [{"==": [{"var": "roadName"}, "SP-249"]}, {">=": [{"var": "km"}, 144.115]}, {"<=": [{"var": "km"}, 149.0]}]}, "Coronel Macedo", {"and": [{"==": [{"var": "roadName"}, "SP-249"]}, {">=": [{"var": "km"}, 149.0]}, {"<=": [{"var": "km"}, 159.0]}]}, "Taquarituba", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 83.2]}, {"<=": [{"var": "km"}, 95.16]}]}, "Araraquara", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 95.16]}, {"<=": [{"var": "km"}, 122.22]}]}, "Boa Esperan\u00e7a do Sul", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 122.22]}, {"<=": [{"var": "km"}, 124.02]}]}, "Dourado", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 124.02]}, {"<=": [{"var": "km"}, 138.61]}]}, "Bocaina", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 138.61]}, {"<=": [{"var": "km"}, 147.3]}]}, "Ja\u00fa", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 155.77]}, {"<=": [{"var": "km"}, 167.02]}]}, "Ja\u00fa", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 167.02]}, {"<=": [{"var": "km"}, 177.8]}]}, "Barra Bonita", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 177.8]}, {"<=": [{"var": "km"}, 186.7]}]}, "Igara\u00e7u do Tiet\u00ea", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 186.7]}, {"<=": [{"var": "km"}, 212.24]}]}, "S\u00e3o Manuel", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 212.24]}, {"<=": [{"var": "km"}, 225.83]}]}, "Prat\u00e2nia", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 225.83]}, {"<=": [{"var": "km"}, 237.06]}]}, "Botucatu", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 237.06]}, {"<=": [{"var": "km"}, 237.77]}]}, "Avar\u00e9", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 288.19]}, {"<=": [{"var": "km"}, 309.04]}]}, "Ita\u00ed", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 309.04]}, {"<=": [{"var": "km"}, 320.0]}]}, "Taquarituba", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 334.25]}, {"<=": [{"var": "km"}, 345.46]}]}, "Coronel Macedo", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 345.46]}, {"<=": [{"var": "km"}, 357.5]}]}, "Itaporanga", {"and": [{"==": [{"var": "roadName"}, "SP-281"]}, {">=": [{"var": "km"}, 43.36]}, {"<=": [{"var": "km"}, 49.64]}]}, "Riversul", {"and": [{"==": [{"var": "roadName"}, "SP-281"]}, {">=": [{"var": "km"}, 49.64]}, {"<=": [{"var": "km"}, 67.72]}]}, "Itaporanga", {"and": [{"==": [{"var": "roadName"}, "SP-281"]}, {">=": [{"var": "km"}, 67.72]}, {"<=": [{"var": "km"}, 70.85]}]}, "Bar\u00e3o de Antonina", {"and": [{"==": [{"var": "roadName"}, "SP-304"]}, {">=": [{"var": "km"}, 285.0]}, {"<=": [{"var": "km"}, 297.0]}]}, "Ja\u00fa", {"and": [{"==": [{"var": "roadName"}, "SPA-043/281"]}, {"<=": [{"var": "km"}, 3.0]}]}, "Riversul", {"and": [{"==": [{"var": "roadName"}, "SPA-085/255"]}, {"<=": [{"var": "km"}, 1.115]}]}, "Araraquara", {"and": [{"==": [{"var": "roadName"}, "SPA-106/255"]}, {"<=": [{"var": "km"}, 7.25]}]}, "Boa Esperan\u00e7a do Sul", {"and": [{"==": [{"var": "roadName"}, "SPA-112/255"]}, {"<=": [{"var": "km"}, 1.1]}]}, "Boa Esperan\u00e7a do Sul", {"and": [{"==": [{"var": "roadName"}, "SPA-114/255"]}, {"<=": [{"var": "km"}, 1.0]}]}, "Boa Esperan\u00e7a do Sul", {"and": [{"==": [{"var": "roadName"}, "SPA-115/255"]}, {"<=": [{"var": "km"}, 2.1]}]}, "Boa Esperan\u00e7a do Sul", {"and": [{"==": [{"var": "roadName"}, "SPA-115/255"]}, {">=": [{"var": "km"}, 2.1]}, {"<=": [{"var": "km"}, 7.2]}]}, "Trabiju", {"and": [{"==": [{"var": "roadName"}, "SPA-133/255"]}, {"<=": [{"var": "km"}, 5.7]}]}, "Bocaina", {"and": [{"==": [{"var": "roadName"}, "SPA-138/255"]}, {"<=": [{"var": "km"}, 4.8]}]}, "Bocaina", {"and": [{"==": [{"var": "roadName"}, "SPA-179/255"]}, {"<=": [{"var": "km"}, 0.4]}]}, "Igara\u00e7u do Tiet\u00ea", {"and": [{"==": [{"var": "roadName"}, "SPA-196/255"]}, {"<=": [{"var": "km"}, 2.5]}]}, "S\u00e3o Manuel", {"and": [{"==": [{"var": "roadName"}, "SPA-198/255"]}, {"<=": [{"var": "km"}, 5.7]}]}, "S\u00e3o Manuel", {"and": [{"==": [{"var": "roadName"}, "SPA-214/255"]}, {"<=": [{"var": "km"}, 0.7]}]}, "Prat\u00e2nia", {"and": [{"==": [{"var": "roadName"}, "SPA-338/255"]}, {"<=": [{"var": "km"}, 0.93]}]}, "Coronel Macedo", {"and": [{"==": [{"var": "roadName"}, "SPI-274/310"]}, {"<=": [{"var": "km"}, 2.1]}]}, "Araraquara", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 2.8]}, {"<=": [{"var": "km"}, 19.0]}]}, "Ribeir\u00e3o Preto", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 19.0]}, {"<=": [{"var": "km"}, 24.25]}]}, "Cravinhos", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 24.25]}, {"<=": [{"var": "km"}, 32.87]}]}, "Lu\u00eds Ant\u00f4nio", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 32.87]}, {"<=": [{"var": "km"}, 47.6]}]}, "Guatapar\u00e1", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 47.6]}, {"<=": [{"var": "km"}, 53.5]}]}, "Rinc\u00e3o", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 53.5]}, {"<=": [{"var": "km"}, 61.1]}]}, "Santa L\u00facia", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 61.1]}, {"<=": [{"var": "km"}, 71.0]}]}, "Am\u00e9rico Brasiliense", {"and": [{"==": [{"var": "roadName"}, "SP-255"]}, {">=": [{"var": "km"}, 71.0]}, {"<=": [{"var": "km"}, 83.2]}]}, "Araraquara", {"and": [{"==": [{"var": "roadName"}, "SP-257"]}, {"<=": [{"var": "km"}, 6.92]}]}, "Am\u00e9rico Brasiliense", {"and": [{"==": [{"var": "roadName"}, "SP-257"]}, {">=": [{"var": "km"}, 6.92]}, {"<=": [{"var": "km"}, 16.1]}]}, "Santa L\u00facia", {"and": [{"==": [{"var": "roadName"}, "SP-257"]}, {">=": [{"var": "km"}, 16.1]}, {"<=": [{"var": "km"}, 19.8]}]}, "Rinc\u00e3o", {"and": [{"==": [{"var": "roadName"}, "SP-318"]}, {">=": [{"var": "km"}, 235.4]}, {"<=": [{"var": "km"}, 267.4]}]}, "S\u00e3o Carlos", {"and": [{"==": [{"var": "roadName"}, "SP-318"]}, {">=": [{"var": "km"}, 267.4]}, {"<=": [{"var": "km"}, 270.5]}]}, "Am\u00e9rico Brasiliense", {"and": [{"==": [{"var": "roadName"}, "SP-318"]}, {">=": [{"var": "km"}, 270.5]}, {"<=": [{"var": "km"}, 271.6]}]}, "Santa L\u00facia", {"and": [{"==": [{"var": "roadName"}, "SP-318"]}, {">=": [{"var": "km"}, 271.6]}, {"<=": [{"var": "km"}, 280.0]}]}, "Rinc\u00e3o", {"and": [{"==": [{"var": "roadName"}, "SP-328"]}, {">=": [{"var": "km"}, 289.83]}, {"<=": [{"var": "km"}, 292.76]}]}, "S\u00e3o Sim\u00e3o", {"and": [{"==": [{"var": "roadName"}, "SP-328"]}, {">=": [{"var": "km"}, 292.76]}, {"<=": [{"var": "km"}, 300.28]}]}, "Cravinhos", {"and": [{"==": [{"var": "roadName"}, "SP-328"]}, {">=": [{"var": "km"}, 300.28]}, {"<=": [{"var": "km"}, 300.964]}]}, "Ribeir\u00e3o Preto", {"and": [{"==": [{"var": "roadName"}, "SP-328"]}, {">=": [{"var": "km"}, 300.964]}, {"<=": [{"var": "km"}, 301.518]}]}, "Cravinhos", {"and": [{"==": [{"var": "roadName"}, "SP-328"]}, {">=": [{"var": "km"}, 301.518]}, {"<=": [{"var": "km"}, 302.06]}]}, "Ribeir\u00e3o Preto", {"and": [{"==": [{"var": "roadName"}, "SP-328"]}, {">=": [{"var": "km"}, 304.31]}, {"<=": [{"var": "km"}, 306.82]}]}, "Ribeir\u00e3o Preto", {"and": [{"==": [{"var": "roadName"}, "SP-328"]}, {">=": [{"var": "km"}, 307.6]}, {"<=": [{"var": "km"}, 311.93]}]}, "Ribeir\u00e3o Preto", {"and": [{"==": [{"var": "roadName"}, "SP-330"]}, {">=": [{"var": "km"}, 240.0]}, {"<=": [{"var": "km"}, 261.0]}]}, "Santa Rita do Passa Quatro", {"and": [{"==": [{"var": "roadName"}, "SP-330"]}, {">=": [{"var": "km"}, 261.0]}, {"<=": [{"var": "km"}, 284.81]}]}, "S\u00e3o Sim\u00e3o", {"and": [{"==": [{"var": "roadName"}, "SP-330"]}, {">=": [{"var": "km"}, 284.81]}, {"<=": [{"var": "km"}, 289.65]}]}, "Cravinhos", {"and": [{"==": [{"var": "roadName"}, "SP-330"]}, {">=": [{"var": "km"}, 289.65]}, {"<=": [{"var": "km"}, 300.34]}]}, "Cravinhos", {"and": [{"==": [{"var": "roadName"}, "SP-330"]}, {">=": [{"var": "km"}, 300.34]}, {"<=": [{"var": "km"}, 319.0]}]}, "Ribeir\u00e3o Preto", {"and": [{"==": [{"var": "roadName"}, "SP-334"]}, {">=": [{"var": "km"}, 318.0]}, {"<=": [{"var": "km"}, 321.0]}]}, "Ribeir\u00e3o Preto", {"and": [{"==": [{"var": "roadName"}, "SP-334"]}, {">=": [{"var": "km"}, 321.0]}, {"<=": [{"var": "km"}, 330.96]}]}, "Jardin\u00f3polis", {"and": [{"==": [{"var": "roadName"}, "SP-334"]}, {">=": [{"var": "km"}, 330.96]}, {"<=": [{"var": "km"}, 342.05]}]}, "Brod\u00f3wski", {"and": [{"==": [{"var": "roadName"}, "SP-334"]}, {">=": [{"var": "km"}, 342.05]}, {"<=": [{"var": "km"}, 372.0]}]}, "Batatais", {"and": [{"==": [{"var": "roadName"}, "SP-334"]}, {">=": [{"var": "km"}, 372.0]}, {"<=": [{"var": "km"}, 388.65]}]}, "Restinga", {"and": [{"==": [{"var": "roadName"}, "SP-334"]}, {">=": [{"var": "km"}, 388.65]}, {"<=": [{"var": "km"}, 409.0]}]}, "Franca", {"and": [{"==": [{"var": "roadName"}, "SP-345"]}, {">=": [{"var": "km"}, 10.5]}, {"<=": [{"var": "km"}, 12.2]}]}, "Itirapu\u00e3", {"and": [{"==": [{"var": "roadName"}, "SP-345"]}, {">=": [{"var": "km"}, 10.0]}, {"<=": [{"var": "km"}, 25.5]}]}, "Patroc\u00ednio Paulista", {"and": [{"==": [{"var": "roadName"}, "SP-345"]}, {">=": [{"var": "km"}, 25.5]}, {"<=": [{"var": "km"}, 40.0]}]}, "Franca", {"and": [{"==": [{"var": "roadName"}, "SPA-004/257"]}, {"<=": [{"var": "km"}, 4.6]}]}, "Am\u00e9rico Brasiliense", {"and": [{"==": [{"var": "roadName"}, "SPA-011/345"]}, {"<=": [{"var": "km"}, 1.8]}]}, "Itirapu\u00e3", {"and": [{"==": [{"var": "roadName"}, "SPA-020/345"]}, {"<=": [{"var": "km"}, 2.5]}]}, "Patroc\u00ednio Paulista", {"and": [{"==": [{"var": "roadName"}, "SPA-051/255"]}, {"<=": [{"var": "km"}, 12.8]}]}, "Rinc\u00e3o", {"and": [{"==": [{"var": "roadName"}, "SPA-074/255"]}, {"<=": [{"var": "km"}, 5.0]}]}, "Araraquara", {"and": [{"==": [{"var": "roadName"}, "SPA-077/255"]}, {"<=": [{"var": "km"}, 3.0]}]}, "Araraquara", {"and": [{"==": [{"var": "roadName"}, "SPA-241/330"]}, {"<=": [{"var": "km"}, 11.95]}]}, "Santa Rita do Passa Quatro", {"and": [{"==": [{"var": "roadName"}, "SPA-268/310"]}, {"<=": [{"var": "km"}, 2.9]}]}, "Araraquara", {"and": [{"==": [{"var": "roadName"}, "SPA-271/310"]}, {"<=": [{"var": "km"}, 0.58]}]}, "Araraquara", {"and": [{"==": [{"var": "roadName"}, "SPA-307/330"]}, {"<=": [{"var": "km"}, 3.6]}]}, "Ribeir\u00e3o Preto", {"and": [{"==": [{"var": "roadName"}, "SPA-318/330"]}, {"<=": [{"var": "km"}, 1.65]}]}, "Ribeir\u00e3o Preto", {"and": [{"==": [{"var": "roadName"}, "SPA-321/334"]}, {"<=": [{"var": "km"}, 4.3]}]}, "Jardin\u00f3polis", {"and": [{"==": [{"var": "roadName"}, "SPA-334/334"]}, {"<=": [{"var": "km"}, 1.469]}]}, "Brod\u00f3wski", {"and": [{"==": [{"var": "roadName"}, "SPA-334/334"]}, {">=": [{"var": "km"}, 1.469]}, {"<=": [{"var": "km"}, 9.7]}]}, "Jardin\u00f3polis", {"and": [{"==": [{"var": "roadName"}, "SPA-348/334"]}, {"<=": [{"var": "km"}, 3.5]}]}, "Batatais", {"and": [{"==": [{"var": "roadName"}, "SPA-352/334"]}, {"<=": [{"var": "km"}, 2.9]}]}, "Batatais", {"and": [{"==": [{"var": "roadName"}, "SPA-384/334"]}, {"<=": [{"var": "km"}, 4.2]}]}, "Restinga", {"and": [{"==": [{"var": "roadName"}, "SPA-397/334"]}, {">=": [{"var": "km"}, 2.44]}, {"<=": [{"var": "km"}, 4.47]}]}, "Franca", {"and": [{"==": [{"var": "roadName"}, "SPA-397/334"]}, {">=": [{"var": "km"}, 4.47]}, {"<=": [{"var": "km"}, 9.95]}]}, "Restinga", "NA"]},
                                        "displayName": "Município"}
                                       )
        ac.form_fields['groups'][0]['members'].append(size+1)
#     for index,group in enumerate(ac.form_fields['groups']):
#         if 'Identificação' in group['displayName']:
#             ac.form_fields['fields'].append({
#                                                 "id": size+1,
#                                                 "apiName": "previousDetection",
#                                                 "dataType": "string",
#                                                 "displayName": "Detecção Prévia"
#                                             })
#             ac.form_fields['groups'][index]['members'].append(size+1)
    ac.save()

  0%|          | 0/30 [00:00<?, ?it/s]

# alteração innerFields

In [8]:
for ac in tqdm(occurrence):
    for arr in ac.form_fields['fields']:
        if 'service' == arr.get('apiName'):
            for inner in arr.get('innerFields'):
                if 'location' == inner.get('apiName'):
                    print(ac.name)
                    break

  0%|          | 0/69 [00:00<?, ?it/s]

Afundamento Da Trilha Consolidado
Afundamento Da Trilha Consolidado ARTESP
Afundamento Da Trilha Plástico
Afundamento da Trilha Plástico ARTESP
Afundamento Local Consolidado
Afundamento Local Consolidado ARTESP
Afundamento Local Plástico
Afundamento Local Plástico ARTESP
Classe em validação
Correção De Degrau
Correção De Degrau ARTESP
Depressão
Depressão ARTESP
Desgaste
Desgaste ARTESP
Desplacamento
Desplacamento ARTESP
Escorregamento
Escorregamento ARTESP
Estrutural
Estrutural ARTESP
Fresagem
Fresagem ARTESP
Fresagem Estrutural
Fresagem Fina
Fresagem Fina ARTESP
Fresagem Fina com Micro Revestimento
Fresagem Fina com Micro Revestimento ARTESP
Fresagem Funcional ARTESP
Microrevestimento
Microrevestimento ARTESP
NP-Não Previsto
Ondulação Ou Corrugação
Ondulação Ou Corrugação ARTESP
Panela
Panela ARTESP
Retrabalho
Revestimento
Revestimento ARTESP
Trinca Interligada "Bloco"
Trinca Interligada "Bloco" ARTESP
Trinca Interligada "Jacaré"
Trinca Interligada "Jacaré" ARTESP
Trinca Isolada
Trinc

In [9]:
for ac in tqdm(occurrence):
    for arr in ac.form_fields['fields']:
        if 'service' == arr.get('apiName'):
            for inner in arr.get('innerFields'):
                if 'location' == inner.get('apiName'):
                    inner.get('selectOptions').get('options').append({'name': 'Duplo', 'value': '7'})
                if 'area' == inner.get('apiName'):
                    inner['autofill']={
                            "if": [
                              {
                                "in": [
                                  {
                                    "var": {
                                      "cat": [
                                        "formData.service.",
                                        {
                                          "var": "_index"
                                        },
                                        ".servicetype"
                                      ]
                                    }
                                  },
                                  [
                                    "60"
                                  ]
                                ]
                              },
                              {
                                "if": [
                                  {
                                    "missing": [
                                      {
                                        "cat": [
                                          "formData.service.",
                                          {
                                            "var": "_index"
                                          },
                                          ".width"
                                        ]
                                      },
                                      {
                                        "cat": [
                                          "formData.service.",
                                          {
                                            "var": "_index"
                                          },
                                          ".length"
                                        ]
                                      }
                                    ]
                                  },
                                  0,
                                  {
                                    "*": [
                                      0.5,
                                      {
                                        "var": {
                                          "cat": [
                                            "formData.service.",
                                            {
                                              "var": "_index"
                                            },
                                            ".width"
                                          ]
                                        }
                                      },
                                      {
                                        "var": {
                                          "cat": [
                                            "formData.service.",
                                            {
                                              "var": "_index"
                                            },
                                            ".length"
                                          ]
                                        }
                                      }
                                    ]
                                  }
                                ]
                              },
                              {
                                "if": [
                                  {
                                    "and": [
                                      {
                                        "in": [
                                          {
                                            "var": {
                                              "cat": [
                                                "formData.service.",
                                                {
                                                  "var": "_index"
                                                },
                                                ".servicetype"
                                              ]
                                            }
                                          },
                                          [
                                            "68"
                                          ]
                                        ]
                                      },
                                      {
                                        "in": [
                                          {
                                            "var": {
                                              "cat": [
                                                "formData.service.",
                                                {
                                                  "var": "_index"
                                                },
                                                ".location"
                                              ]
                                            }
                                          },
                                          [
                                            "2",
                                            "3",
                                            "4","7"
                                          ]
                                        ]
                                      }
                                    ]
                                  },
                                  {
                                    "if": [
                                      {
                                        "missing": [
                                          {
                                            "cat": [
                                              "formData.service.",
                                              {
                                                "var": "_index"
                                              },
                                              ".width"
                                            ]
                                          },
                                          {
                                            "cat": [
                                              "formData.service.",
                                              {
                                                "var": "_index"
                                              },
                                              ".length"
                                            ]
                                          },
                                          {
                                            "cat": [
                                              "formData.service.",
                                              {
                                                "var": "_index"
                                              },
                                              ".frequency"
                                            ]
                                          }
                                        ]
                                      },
                                      0,
                                      {
                                        "*": [
                                          {
                                            "var": {
                                              "cat": [
                                                "formData.service.",
                                                {
                                                  "var": "_index"
                                                },
                                                ".width"
                                              ]
                                            }
                                          },
                                          {
                                            "var": {
                                              "cat": [
                                                "formData.service.",
                                                {
                                                  "var": "_index"
                                                },
                                                ".length"
                                              ]
                                            }
                                          },
                                          {
                                            "var": {
                                              "cat": [
                                                "formData.service.",
                                                {
                                                  "var": "_index"
                                                },
                                                ".frequency"
                                              ]
                                            }
                                          }
                                        ]
                                      }
                                    ]
                                  },
                                  {
                                    "if": [
                                      {
                                        "in": [
                                          {
                                            "var": {
                                              "cat": [
                                                "formData.service.",
                                                {
                                                  "var": "_index"
                                                },
                                                ".servicetype"
                                              ]
                                            }
                                          },
                                          [
                                            "52",
                                            "54"
                                          ]
                                        ]
                                      },
                                      {
                                        "if": [
                                          {
                                            "missing": [
                                              {
                                                "cat": [
                                                  "formData.service.",
                                                  {
                                                    "var": "_index"
                                                  },
                                                  ".width"
                                                ]
                                              },
                                              {
                                                "cat": [
                                                  "formData.service.",
                                                  {
                                                    "var": "_index"
                                                  },
                                                  ".length"
                                                ]
                                              },
                                              {
                                                "cat": [
                                                  "formData.service.",
                                                  {
                                                    "var": "_index"
                                                  },
                                                  ".layerquantity"
                                                ]
                                              }
                                            ]
                                          },
                                          0,
                                          {
                                            "*": [
                                              {
                                                "var": {
                                                  "cat": [
                                                    "formData.service.",
                                                    {
                                                      "var": "_index"
                                                    },
                                                    ".width"
                                                  ]
                                                }
                                              },
                                              {
                                                "var": {
                                                  "cat": [
                                                    "formData.service.",
                                                    {
                                                      "var": "_index"
                                                    },
                                                    ".length"
                                                  ]
                                                }
                                              },
                                              {
                                                "var": {
                                                  "cat": [
                                                    "formData.service.",
                                                    {
                                                      "var": "_index"
                                                    },
                                                    ".layerquantity"
                                                  ]
                                                }
                                              }
                                            ]
                                          }
                                        ]
                                      },
                                      {
                                        "if": [
                                          {
                                            "missing": [
                                              {
                                                "cat": [
                                                  "formData.service.",
                                                  {
                                                    "var": "_index"
                                                  },
                                                  ".width"
                                                ]
                                              },
                                              {
                                                "cat": [
                                                  "formData.service.",
                                                  {
                                                    "var": "_index"
                                                  },
                                                  ".length"
                                                ]
                                              }
                                            ]
                                          },
                                          0,
                                          {
                                            "*": [
                                              {
                                                "var": {
                                                  "cat": [
                                                    "formData.service.",
                                                    {
                                                      "var": "_index"
                                                    },
                                                    ".width"
                                                  ]
                                                }
                                              },
                                              {
                                                "var": {
                                                  "cat": [
                                                    "formData.service.",
                                                    {
                                                      "var": "_index"
                                                    },
                                                    ".length"
                                                  ]
                                                }
                                              }
                                            ]
                                          }
                                        ]
                                      }
                                    ]
                                  }
                                ]
                              }
                            ]
                          }
                if 'frequency' == inner.get('apiName'):
                    inner['displayIf']={
                            "in": [
                              {
                                "var": {
                                  "cat": [
                                    "data.formData.service",
                                    {
                                      "var": "index.0"
                                    },
                                    ".location"
                                  ]
                                }
                              },
                              [
                                "2",
                                "3",
                                "4","7"
                              ]
                            ]
                          }
                    inner['requiredIf'] = {
                            "in": [
                              {
                                "var": {
                                  "cat": [
                                    "data.formData.service",
                                    {
                                      "var": "index.0"
                                    },
                                    ".location"
                                  ]
                                }
                              },
                              [
                                "2",
                                "3",
                                "4","7"
                              ]
                            ]
                          }
                    inner['autofill'] = {
                                          "if": [
                                            {
                                              "and": [
                                                {
                                                  "in": [
                                                    {
                                                      "var": {
                                                        "cat": [
                                                          "formData.service.",
                                                          {
                                                            "var": "_index"
                                                          },
                                                          ".servicetype"
                                                        ]
                                                      }
                                                    },
                                                    [
                                                      "68"
                                                    ]
                                                  ]
                                                },
                                                {
                                                  "in": [
                                                    {
                                                      "var": {
                                                        "cat": [
                                                          "formData.service.",
                                                          {
                                                            "var": "_index"
                                                          },
                                                          ".location"
                                                        ]
                                                      }
                                                    },
                                                    [
                                                      "7"
                                                    ]
                                                  ]
                                                }
                                              ]
                                            },2
                                          ]
                                        }
    ac.save()

  0%|          | 0/69 [00:00<?, ?it/s]

In [7]:
ac.form_fields['fields'],ac.name

([{'id': 1,
   'unit': 'm',
   'apiName': 'length',
   'dataType': 'float',
   'displayName': 'Comprimento'},
  {'id': 2,
   'unit': 'm',
   'apiName': 'width',
   'dataType': 'float',
   'displayName': 'Largura'},
  {'id': 3,
   'unit': 'cm',
   'apiName': 'height',
   'dataType': 'float',
   'displayName': 'Espessura'},
  {'id': 4,
   'unit': 'm²',
   'apiName': 'area',
   'autofill': {'if': [{'missing': ['formData.width', 'formData.length']},
     0,
     {'*': [{'var': 'formData.width'}, {'var': 'formData.length'}]}]},
   'dataType': 'float',
   'disableIf': True,
   'displayName': 'Área'},
  {'id': 5,
   'unit': 'm³',
   'apiName': 'volume',
   'autofill': {'if': [{'missing': ['formData.width',
       'formData.length',
       'formData.height']},
     0,
     {'*': [0.01,
       {'var': 'formData.width'},
       {'var': 'formData.length'},
       {'var': 'formData.height'}]}]},
   'dataType': 'float',
   'disableIf': True,
   'displayName': 'Volume'},
  {'id': 6,
   'apiName': 'a

In [54]:
for ac in tqdm(occurrence.exclude(name__in=[a.name for a in tem_id])):
    size = len(ac.form_fields['fields'])
   
    ac.form_fields['fields'].append({
                "id": size+1,
                "apiName": "responsibleTeam",
                "dataType": "select",
                "displayName": "Responsável",
                "selectOptions": {
                    "options": [
                        {
                            "name": "Conserva",
                            "value": "1"
                        },
                        {
                            "name": "Engenharia",
                            "value": "2"
                        },
                        {
                            "name": "TI",
                            "value": "3"
                        },
                        {
                            "name": "Facilities",
                            "value": "4"
                        }
                    ]
                }
            })
    ac.form_fields['groups'].insert(0,{'order': 1, 'members': [size+1], 'displayName': 'Identificação'})
    for a in ac.form_fields['groups'][1:]:
        a['order'] = a['order']+1
    ac.save()

  0%|          | 0/47 [00:00<?, ?it/s]

In [15]:
ac.form_fields['groups']

[{'order': 1, 'members': [1, 2, 3, 4, 5, 8], 'displayName': 'Quantitativos'},
 {'order': 2, 'members': [7, 6], 'displayName': 'Informações adicionais'}]

# Verifica se campo ja existe no form

In [16]:
contador=0
for ac in tqdm(occurrence):
    sever=False
    for arr in ac.form_fields['fields']:
        if "demandOrigin" != arr.get('apiName'):
            sever=True
    if sever:
        contador+=1
        size = len(ac.form_fields['fields'])
        ac.form_fields['fields'].append({
                                        "id": size+1,
                                        "apiName": "demandOrigin",
                                        "dataType": "selectMultiple",
                                        "requiredIf": {
                                            "!": {
                                                "var": "value"
                                            }
                                        },
                                        "displayName": "Origem",
                                        "selectOptions": {
                                            "options": [{
                                                "name": "CCO",
                                                "value": "1"
                                            }, {
                                                "name": "Entrevias",
                                                "value": "2"
                                            }, {
                                                "name": "ARTESP DIN",
                                                "value": "3"
                                            }, {
                                                "name": "ARTESP DOP",
                                                "value": "4"
                                            }]
                                        }
                                    })
        ac.form_fields['fields'].append({
                                        "id": size+2,
                                        "apiName": "ccoCode",
                                        "dataType": "string",
                                        "displayIf": {
                                            "in": ["1", {
                                                "var": "data.formData.demandOrigin"
                                            }]
                                        },
                                        "displayName": "C\u00f3digo CCO"
                                    })
        ac.form_fields['fields'].append({
                                        "id": size+3,
                                        "apiName": "internCode",
                                        "dataType": "string",
                                        "displayIf": {
                                            "in": ["2", {
                                                "var": "data.formData.demandOrigin"
                                            }]
                                        },
                                        "displayName": "C\u00f3digo Interno"
                                    })
        ac.form_fields['fields'].append({
                                        "id": size+4,
                                        "apiName": "artespCode",
                                        "dataType": "string",
                                        "displayIf": {
                                            "or": [{
                                                "in": ["3", {
                                                    "var": "data.formData.demandOrigin"
                                                }]
                                            }, {
                                                "in": ["4", {
                                                    "var": "data.formData.demandOrigin"
                                                }]
                                            }]
                                        },
                                        "displayName": "C\u00f3digo da ocorr\u00eancia"
                                    })
        ac.form_fields['fields'].append({
                                        "id": size+5,
                                        "apiName": "receivedDate",
                                        "dataType": "timestamp",
                                        "displayIf": {
                                            "or": [{
                                                "in": ["3", {
                                                    "var": "data.formData.demandOrigin"
                                                }]
                                            }, {
                                                "in": ["4", {
                                                    "var": "data.formData.demandOrigin"
                                                }]
                                            }]
                                        },
                                        "displayName": "Recebido da ARTESP em:"
                                    })
        
#         counter=len(ac.form_fields['groups'])-1
        
        ac.form_fields['groups'][0]['members'].append(size+1)
        ac.form_fields['groups'][0]['members'].append(size+2)
        ac.form_fields['groups'][0]['members'].append(size+3)
        ac.form_fields['groups'][0]['members'].append(size+4)
        ac.form_fields['groups'][0]['members'].append(size+5)
        
        ac.save()
contador

  0%|          | 0/33 [00:00<?, ?it/s]

33

In [12]:
occurrence[2],occurrence[2].form_fields['fields'],occurrence[2].form_fields['groups']

(<OccurrenceType: AMP Cata papel - ['Entrevias']>,
 [{'id': 1,
   'apiName': 'ampCode',
   'dataType': 'string',
   'displayName': 'Código de controle'},
  {'id': 2,
   'apiName': 'length',
   'dataType': 'float',
   'displayName': 'Comprimento (m)'},
  {'id': 3,
   'apiName': 'width',
   'dataType': 'float',
   'displayName': 'Largura (m)'},
  {'id': 4,
   'apiName': 'height',
   'dataType': 'float',
   'displayName': 'Altura/Espessura (m)'},
  {'id': 5,
   'logic': OrderedDict([('if',
                 [OrderedDict([('missing',
                                ['formData.length', 'formData.width'])]),
                  0,
                  OrderedDict([('*',
                                [OrderedDict([('var', 'formData.length')]),
                                 OrderedDict([('var',
                                               'formData.width')])])])])]),
   'apiName': 'area',
   'dataType': 'jsonLogic',
   'displayName': 'Área (m²)'},
  {'id': 6,
   'unit': 'm³',
   'logic': Orde

# procurar tipo de campo fields das classes

In [19]:
occurrence = OccurrenceType.objects.filter(company__name='Pavicolet (Amazonas)')

In [6]:
for match in tqdm(occurrence):
    for index, field in enumerate(match.form_fields['fields']):
        match.form_fields['fields'][index]=format_keys(keys_to_camel_case(field))
    match.save()

  0%|          | 0/183 [00:00<?, ?it/s]

In [20]:
for ac in tqdm(occurrence):
    existe_reg_ceo = any(a.get('displayName') == 'Diretoria Regional' for a in ac.form_fields['fields'])
    size_fields=len(ac.form_fields['fields'])
    for i,group in enumerate(ac.form_fields['groups']):
        if 'Características' in group.get('displayName') and not existe_reg_ceo:
            ac.form_fields['fields'].append({
                  "id": size_fields+1,
                  "apiName": "regCeo",
                  "dataType": "select",
                  "requiredIf": True,
                  "displayName": "Diretoria Regional",
                  "selectOptions": {
                    "options": [
                      {
                        "name": "DR.1",
                        "value": "1"
                      },
                      {
                        "name": "DR.2",
                        "value": "2"
                      },
                      {
                        "name": "DR.3",
                        "value": "3"
                      },
                      {
                        "name": "DR.4",
                        "value": "4"
                      },
                      {
                        "name": "DR.5",
                        "value": "5"
                      },
                      {
                        "name": "DR.6",
                        "value": "6"
                      },
                      {
                        "name": "DR.7",
                        "value": "7"
                      },
                      {
                        "name": "DR.8",
                        "value": "8"
                      },
                      {
                        "name": "DR.9",
                        "value": "9"
                      },
                      {
                        "name": "DR.10",
                        "value": "10"
                      },
                      {
                        "name": "DR.11",
                        "value": "11"
                      },
                      {
                        "name": "DR.12",
                        "value": "12"
                      },
                      {
                        "name": "DR.13",
                        "value": "13"
                      },
                      {
                        "name": "DR.14",
                        "value": "14"
                      }
                    ]
                  }
                })
            ac.form_fields['groups'][i]['members'].append(size_fields+1)
    
    ac.save()   

  0%|          | 0/92 [00:00<?, ?it/s]

In [17]:
ac.form_fields['displayName']

'Veículos DER'

In [16]:
ac.save()

In [16]:
occurrence[0].form_fields['fields'][6]

{'id': 7,
 'apiName': 'internCode',
 'dataType': 'string',
 'displayIf': {'or': [{'in': ['3', {'var': 'data.formData.demandOrigin'}]},
   {'in': ['1', {'var': 'data.formData.demandOrigin'}]}]},
 'displayName': 'Cód interno'}

In [32]:
for ac in tqdm(occurrence):
    size=len(ac.form_fields['fields'])
    cond=False
    for b in ac.form_fields['fields']:
        if "notesField" in b["apiName"]:
            cond=False
        else:
            cond=True
    if cond:
        ac.form_fields['fields'].append({
                                        "id": size+1,
                                        "apiName": "notesField",
                                        "dataType": "textArea",
                                        "displayName": "Observações de campo",
                                    })

        for a in ac.form_fields['groups']:
            if 'Quantitativo' in a.get('displayName'):
                a['members'].append(size+1)
            try:
                if 'Quantitativo' in a.get('display_name'):
                    a['members'].append(size+1)
            except:
                pass
    ac.save()   

  0%|          | 0/22 [00:00<?, ?it/s]

# Deletar o ultimo item de uma lista recem adicionado

In [31]:
for ac in tqdm(occurrence):
    del ac.form_fields['fields'][-1]
#     for arr in ac.form_fields['fields']:
    for a in ac.form_fields['groups']:
        if 'Quantitativo' in a.get('displayName'):
            del a['members'][-1]
        try:
            if 'Quantitativo' in a.get('display_name'):
                del a['members'][-1]
        except:
            pass
    ac.save()

  0%|          | 0/22 [00:00<?, ?it/s]